In [6]:
# Ich benutze hier den Kernel aus dem mamba tango enviroment.

import asyncio, time, nest_asyncio
nest_asyncio.apply()

# Notizen zu asyncio
## Links
https://cheat.readthedocs.io/en/latest/python/asyncio.html

https://docs.python.org/3/library/asyncio-task.html

https://www.youtube.com/watch?v=HzpdMDYEstA

## Nutzen 
- Geradeliniger Code
- einfacher zu Handhaben als Threads, weil keine echte Parallelität
- Schlanker als Threads, 3 kB vs 50 kB  

## Eventloop
der asyncio-Code kann nur in einer Event Loop laufen. Die Schleife ist der Treiber, der das kooperative Multitasking managt.

## Coroutines

Python unterscheidet zw. Cor. Funktion und Cor. Objekt
- `async def` definiert eine CF. 
- Nur CF kann `await` benutzen
- Aufrufen einer CF liefert ein Objekt: 
```python
async def coro_f():
    print("Corouteine, schläft 1s")
    asyncio.sleep(1)
    print("Ausgeschlafen")

cf=coro_f() #keine Ausführung
```
## Futures
F ist ein Objekt, die zukünftige Werte oder Ausnahmen repräsentiert. F Objekts werden zur Verwaltung asynchroner Operationen genutzt. Sehe auch Vergeich mit Tasks.
F können Callback Funktionen aufrufen.


In [3]:
async def perform_async_operation(future):
    # Simuliere eine asynchrone Operation, z.B. eine Berechnung oder Abfrage
    await asyncio.sleep(2)
    # Setze das Ergebnis des Futures nach Abschluss der Operation
    future.set_result("Operation completed successfully!")

def future_cb1(future):
    print("Ich bin die 1. CallBack Funktion!")

def future_cb2(future):
    print("Ich bin die 2. CallBack Funktion!")

future = asyncio.Future()
future.add_done_callback(future_cb1) # DONE CALLBACK
future.add_done_callback(future_cb2)

await perform_async_operation(future)
result = await future
print(result)

Operation completed successfully!
Ich bin die 1. CallBack Funktion!
Ich bin die 2. CallBack Funktion!


In [8]:
async def set_future_value(future, value):
    # Simuliere eine asynchrone Operation
    await asyncio.sleep(1)
    future.set_result(value) #!!! Dieser Wert kann später abgefragt werden!!!

async def main():
    # Erstellen eines Future-Objekts
    future = asyncio.Future()
    
    # Starte eine Aufgabe, die den Future-Wert setzt
    asyncio.create_task(set_future_value(future, 'Hello, Future!'))

    # Warten auf das Setzen des Future-Werts
    result = await future
    print(result)

# Startet das asyncio-Event-Loop und führt die main()-Funktion aus
asyncio.run(main())


Hello, Future!


## Tasks
Ts planen und verwalten die Ausführung von CFs. 
- `mytask = asyncio.create_task(coro_f(params))` `create_task` bekommt eine CF und gibt ein TO zu steuerung des Tasks.
- Ts werden **unmittelbar** nach erstellung ausgeführt!

In [18]:
async def say_hello():
    print("*** 2. Hello, I am the Task! ***")
    await asyncio.sleep(2)
    print("*** 5. I have finished! ***")

async def main():

    print("1. Task erstellen und ausführen") 
    task = asyncio.create_task(say_hello())
    # await asyncio.sleep(0.1)
    print("3. Doing something while waiting...") # ACHTUNG, ohne await davor, wird diese Zeile VOR Task ausgegeben. 
    await asyncio.sleep(0.9)
    print(" 4. Doing something else while waiting...")
    
    # Warten auf die Task
    await task
    print("6. Doing something after Task has finished.")


# Startet den asyncio-Event-Loop und führt main() aus
asyncio.run(main())

1. Task erstellen und ausführen
3. Doing something while waiting...
*** 2. Hello, I am the Task! ***
 4. Doing something else while waiting...
*** 5. I have finished! ***
6. Doing something after Task has finished.


In [32]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

async def main():
    print(f"started at {time.strftime('%X')}")

    task1 = asyncio.create_task(
        say_after(1, 'hello'))

    task2 = asyncio.create_task(
        say_after(2, 'world'))

    await task1
    await task2
    
    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

started at 11:18:50
hello
world
finished at 11:18:52



### Task vs. Futures:
Task ist eine Kindklasse von Future, erbt also alles.
- Futures sind allgemeine Platzhalter für zukünftige Werte
- Task sind speziell für CF Ausführnng gedacht
- Futures müssen explizit gestartet werden mit `set_result()` oder `set_exception()`
- Task werden verwendet, eine CF in Hintegrund auszuführen
- Future wird oft verwendet, um eine Schnittstelle zu einem System zu definieren, das einen zukünftigen Wert liefert, den man manuell kontrollieren muss, wann und wie er bereitgestellt wird



### Taksgroup
`asyncio.TaskGroup` ist eine moderne (ab Python 3.11) Alternative zu `create_task()` und `asyncio.gather()`

In [33]:
async def main():
    async with asyncio.TaskGroup() as tg:
        task1 = tg.create_task(
            say_after(1, 'hello'))

        task2 = tg.create_task(
            say_after(2, 'world'))

        print(f"started at {time.strftime('%X')}")

    # The await is implicit when the context manager exits.

    print(f"finished at {time.strftime('%X')}")

asyncio.run(main())

started at 11:25:26
hello
world
finished at 11:25:28


In [12]:
async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({number}), currently i={i}...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")
    return f

async def main():
    # Schedule three calls *concurrently*:
    L = await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )
    print(L)

asyncio.run(main())

Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=2...
Task C: Compute factorial(4), currently i=2...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3...
Task C: Compute factorial(4), currently i=3...
Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4...
Task C: factorial(4) = 24
[2, 6, 24]


## Timeouts

In [16]:
async def main():
    try:
        async with asyncio.timeout(20):
            # Schedule three calls *concurrently*:
            L = await asyncio.gather(
                factorial("A", 2),
                factorial("B", 3),
                factorial("C", 4),
            )
        print(L)
    except TimeoutError:
        print("Einige Faktorials konnten nicht berechnet werden!")
    
asyncio.run(main())

Task A: Compute factorial(2), currently i=2...
Task B: Compute factorial(3), currently i=2...
Task C: Compute factorial(4), currently i=2...
Task A: factorial(2) = 2
Task B: Compute factorial(3), currently i=3...
Task C: Compute factorial(4), currently i=3...
Task B: factorial(3) = 6
Task C: Compute factorial(4), currently i=4...
Task C: factorial(4) = 24
[2, 6, 24]


In [17]:
async def wait_for_event(event):
    print("schlaffe")
    #await asyncio.sleep(10)
    try:
        await asyncio.wait_for(event.wait(), timeout=10.0)
        print("Event was set!")
    except asyncio.TimeoutError:
        print("Timeout reached, event was not set.")

event = None
async def main():
    global event 
    event = asyncio.Event()

    # Start the coroutine to wait for the event
    asyncio.create_task(wait_for_event(event))   # Das ist der Grund, warum der Aufruf nichtblockierend ist!

# Run the main function
asyncio.run(main())


schlaffe
Timeout reached, event was not set.


In [10]:
# Set the event (uncomment to set the event and see different behavior)
event.set()